In [1]:
import pandas as pd
import numpy as np
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from keras.optimizers import Adam
from tensorflow.keras.layers import Dense, LSTM
from keras.layers import  Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from datetime import time
data = pd.read_excel('RS_KSR_WFM_STAT_2025_05_14.xlsx')
pd.set_option('display.max_rows', 100)

2025-05-20 05:35:42.607372: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-20 05:35:42.614501: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-20 05:35:42.625372: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747719342.643801 3997731 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747719342.649169 3997731 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747719342.662714 3997731 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
data.columns = ['conversation_start_interval_tmst', 'Time', 'offered', 'actans',
       'actabn', 'absActHt', 'absActSa', 'parent', 'child', 'fiscalDate',
       'fiscalYear', 'ficalQuarter', 'fiscalMonth', 'fiscalWeek', 'DOW']

In [3]:
data = data[~data .offered.isna()]

In [4]:
data = data[data .offered != 0]

In [5]:
data = data[data.conversation_start_interval_tmst<=pd.to_datetime('2025-01-15 00:00:00')]

In [6]:
# 假设数据有两列：'timestamp' 和 'value'
holiday_data = pd.read_excel(r'holiday.xlsx')
holiday_data['~is_holiday'] = 0
Holiday_name = ['Christmas Day', 'Columbus Day',
       'Independence Day', 'Labor Day', 'Martin Luther King Jr. Day',
       'Memorial Day', "New Year's Day", "Presidents' Day", 'Thanksgiving Day',
       'Veterans Day']
holiday_data['date'] = holiday_data.Date.dt.date
data['datetime'] = pd.to_datetime(data.conversation_start_interval_tmst)
data['date'] = data.datetime.dt.date
holiday_datetime = holiday_data.date.to_numpy()
data.datetime = data.datetime.dt.floor('T')
data['datetime'] = data['datetime'].apply(
    lambda x: x.ceil('30T') if x.minute == 29 else x
)
data['datetime'] = data['datetime'].apply(
    lambda x: x.ceil('H') if x.minute == 59 else x
)
data['time'] = data.datetime.dt.time
data = data.sort_values('datetime').reset_index(drop = True)
data = data.sort_values(by='datetime')

# 获取所有唯一的时间点（如每天的09:00, 09:30等）
unique_times = data['time'].unique()

# 获取所有唯一的日期
unique_dates = data['date'].unique()
data = pd.merge(data,holiday_data,on = 'date',how = 'left').fillna(1)
data = data[data['~is_holiday'] == 1]

/tmp/ipykernel_3997731/2301794343.py:12: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  data.datetime = data.datetime.dt.floor('T')


In [7]:
unique_times = [ time(8, 0), time(8, 30),
time(9, 0), time(9, 30), time(10, 0),
time(10, 30), time(11, 0), time(11, 30),
time(12, 0), time(12, 30), time(13, 0),
time(13, 30), time(14, 0), time(14, 30),
time(15, 0), time(15, 30), time(16, 0),
time(16, 30), time(17, 0), time(17, 30),
time(18, 0), time(18, 30), time(19, 0)]

In [8]:
start_time = 35+23 # 58 how many days in advance
end_time = start_time+36 #36 is the train length

In [9]:
add_column = ['actans','actabn','absActHt','absActSa']

In [10]:
add_columns =  [f'actans_{i}' for i in range(start_time+1,end_time+1) ] +\
                [f'actabn_{i}' for i in range(start_time+1,end_time+1) ] +\
                [f'absActHt_{i}' for i in range(start_time+1,end_time+1) ] +\
                [f'absActSa_{i}' for i in range(start_time+1,end_time+1) ]

In [11]:
matrix = []

# 遍历每个时间点
for time,time1,time2 in zip(unique_times,np.roll(unique_times, shift=-1),np.roll(unique_times, shift=1)):
    # filter the 
    time_data = data[data['time'] == time].set_index('date')['offered']    
    full_dates = pd.date_range(start=unique_dates.min(), end=unique_dates.max(), freq='B')  # 仅工作日
    full_dates = full_dates.difference(holiday_datetime)
    time_data = time_data.groupby(time_data.index).sum()
    time_data = time_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
    shifted_data = pd.concat([time_data.shift(i) for i in range(start_time,end_time)], axis=1)
 

In [13]:
   
    
    add_data = data[data['time'] == time].set_index('date')[add_column]
    add_data = add_data.groupby(add_data.index).sum()
    add_data = add_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
    add_data_ = pd.concat([add_data.shift(i) for i in range(start_time,end_time)], axis=1)
    
    
    time_data1 = data[data['time'] == time1].set_index('date')['offered']    
    time_data1 = time_data1.groupby(time_data1.index).sum()
    time_data1 = time_data1.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
    shifted_data1 = pd.concat([time_data1.shift(i) for i in range(start_time,end_time)], axis=1)
    
    time_data2 = data[data['time'] == time2].set_index('date')['offered']    
    time_data2 = time_data2.groupby(time_data2.index).sum()
    time_data2 = time_data2.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
    shifted_data2 = pd.concat([time_data2.shift(i) for i in range(start_time,end_time)], axis=1)

In [14]:
    
    
    year_data = time_data.shift(251)

In [18]:
time_data

2021-12-27    18.0
2021-12-28    15.0
2021-12-29    17.0
2021-12-30    13.0
2021-12-31     2.0
              ... 
2025-01-08    11.0
2025-01-09    17.0
2025-01-10    11.0
2025-01-13    25.0
2025-01-14    20.0
Name: offered, Length: 768, dtype: float64

In [17]:
    

    # 将当前时间点、前半小时和后半小时的数据拼接
    combined_data = pd.concat([time_data,add_data_, shifted_data,shifted_data1,shifted_data2,year_data], axis=1)
 

In [22]:
pd.set_option('display.max_columns',None)
combined_data

,offered,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,actans,actabn,absActHt,absActSa,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered,offered
2021-12-27,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-12-28,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [23]:
   
    # 创建日期+时间列
    datetime_column = np.array([pd.Timestamp(date) + pd.Timedelta(hours=time.hour, minutes=time.minute) for date in full_dates]).reshape(-1, 1)
    

In [24]:
datetime_column

array([[Timestamp('2021-12-27 19:00:00')],
       [Timestamp('2021-12-28 19:00:00')],
       [Timestamp('2021-12-29 19:00:00')],
       [Timestamp('2021-12-30 19:00:00')],
       [Timestamp('2021-12-31 19:00:00')],
       [Timestamp('2022-01-03 19:00:00')],
       [Timestamp('2022-01-04 19:00:00')],
       [Timestamp('2022-01-05 19:00:00')],
       [Timestamp('2022-01-06 19:00:00')],
       [Timestamp('2022-01-07 19:00:00')],
       [Timestamp('2022-01-10 19:00:00')],
       [Timestamp('2022-01-11 19:00:00')],
       [Timestamp('2022-01-12 19:00:00')],
       [Timestamp('2022-01-13 19:00:00')],
       [Timestamp('2022-01-14 19:00:00')],
       [Timestamp('2022-01-18 19:00:00')],
       [Timestamp('2022-01-19 19:00:00')],
       [Timestamp('2022-01-20 19:00:00')],
       [Timestamp('2022-01-21 19:00:00')],
       [Timestamp('2022-01-24 19:00:00')],
       [Timestamp('2022-01-25 19:00:00')],
       [Timestamp('2022-01-26 19:00:00')],
       [Timestamp('2022-01-27 19:00:00')],
       [Tim

In [25]:
    # 将日期+时间列添加到数据中
    combined_data_with_datetime = np.hstack([datetime_column,combined_data.to_numpy()])
    

In [26]:
combined_data_with_datetime

array([[Timestamp('2021-12-27 19:00:00'), 18.0, nan, ..., nan, nan, nan],
       [Timestamp('2021-12-28 19:00:00'), 15.0, nan, ..., nan, nan, nan],
       [Timestamp('2021-12-29 19:00:00'), 17.0, nan, ..., nan, nan, nan],
       ...,
       [Timestamp('2025-01-10 19:00:00'), 11.0, 19.0, ..., 21.0, 29.0,
        16.0],
       [Timestamp('2025-01-13 19:00:00'), 25.0, 19.0, ..., 18.0, 21.0,
        21.0],
       [Timestamp('2025-01-14 19:00:00'), 20.0, 9.0, ..., 25.0, 18.0,
        6.0]], dtype=object)

In [27]:
    # 将结果存入矩阵
    matrix.append(combined_data_with_datetime)

In [28]:
matrix

[array([[Timestamp('2021-12-27 19:00:00'), 18.0, nan, ..., nan, nan, nan],
        [Timestamp('2021-12-28 19:00:00'), 15.0, nan, ..., nan, nan, nan],
        [Timestamp('2021-12-29 19:00:00'), 17.0, nan, ..., nan, nan, nan],
        ...,
        [Timestamp('2025-01-10 19:00:00'), 11.0, 19.0, ..., 21.0, 29.0,
         16.0],
        [Timestamp('2025-01-13 19:00:00'), 25.0, 19.0, ..., 18.0, 21.0,
         21.0],
        [Timestamp('2025-01-14 19:00:00'), 20.0, 9.0, ..., 25.0, 18.0,
         6.0]], dtype=object)]

In [29]:
max_rows = max(arr.shape[0] for arr in matrix)
max_rows

768

In [30]:
# 将每个时间点的数据填充到最大行数
for i in range(len(matrix)):
    if matrix[i].shape[0] < max_rows:
        padding = np.full((max_rows - matrix[i].shape[0], 11), np.nan)  # 用NaN填充（10列数据 + 1列时间）
        matrix[i] = np.vstack([matrix[i], padding])

In [31]:
matrix

[array([[Timestamp('2021-12-27 19:00:00'), 18.0, nan, ..., nan, nan, nan],
        [Timestamp('2021-12-28 19:00:00'), 15.0, nan, ..., nan, nan, nan],
        [Timestamp('2021-12-29 19:00:00'), 17.0, nan, ..., nan, nan, nan],
        ...,
        [Timestamp('2025-01-10 19:00:00'), 11.0, 19.0, ..., 21.0, 29.0,
         16.0],
        [Timestamp('2025-01-13 19:00:00'), 25.0, 19.0, ..., 18.0, 21.0,
         21.0],
        [Timestamp('2025-01-14 19:00:00'), 20.0, 9.0, ..., 25.0, 18.0,
         6.0]], dtype=object)]

In [32]:
# 将矩阵堆叠成一个大的二维数组
matrix = np.vstack(matrix)

In [38]:
matrix

,datetime,offered,actans_59,actans_60,actans_61,actans_62,actans_63,actans_64,actans_65,actans_66,actans_67,actans_68,actans_69,actans_70,actans_71,actans_72,actans_73,actans_74,actans_75,actans_76,actans_77,actans_78,actans_79,actans_80,actans_81,actans_82,actans_83,actans_84,actans_85,actans_86,actans_87,actans_88,actans_89,actans_90,actans_91,actans_92,actans_93,actans_94,actabn_59,actabn_60,actabn_61,actabn_62,actabn_63,actabn_64,actabn_65,actabn_66,actabn_67,actabn_68,actabn_69,actabn_70,actabn_71,actabn_72,actabn_73,actabn_74,actabn_75,actabn_76,actabn_77,actabn_78,actabn_79,actabn_80,actabn_81,actabn_82,actabn_83,actabn_84,actabn_85,actabn_86,actabn_87,actabn_88,actabn_89,actabn_90,actabn_91,actabn_92,actabn_93,actabn_94,absActHt_59,absActHt_60,absActHt_61,absActHt_62,absActHt_63,absActHt_64,absActHt_65,absActHt_66,absActHt_67,absActHt_68,absActHt_69,absActHt_70,absActHt_71,absActHt_72,absActHt_73,absActHt_74,absActHt_75,absActHt_76,absActHt_77,absActHt_78,absActHt_79,absActHt_80,absActHt_81,absActHt_82,absActHt_83,absActHt_84,absActHt_85,absActHt_86,absActHt_87,absActHt_88,absActHt_89,absActHt_90,absActHt_91,absActHt_92,absActHt_93,absActHt_94,absActSa_59,absActSa_60,absActSa_61,absActSa_62,absActSa_63,absActSa_64,absActSa_65,absActSa_66,absActSa_67,absActSa_68,absActSa_69,absActSa_70,absActSa_71,absActSa_72,absActSa_73,absActSa_74,absActSa_75,absActSa_76,absActSa_77,absActSa_78,absActSa_79,absActSa_80,absActSa_81,absActSa_82,absActSa_83,absActSa_84,absActSa_85,absActSa_86,absActSa_87,absActSa_88,absActSa_89,absActSa_90,absActSa_91,absActSa_92,absActSa_93,absActSa_94,freq_59,freq_60,freq_61,freq_62,freq_63,freq_64,freq_65,freq_66,freq_67,freq_68,freq_69,freq_70,freq_71,freq_72,freq_73,freq_74,freq_75,freq_76,freq_77,freq_78,freq_79,freq_80,freq_81,freq_82,freq_83,freq_84,freq_85,freq_86,freq_87,freq_88,freq_89,freq_90,freq_91,freq_92,freq_93,freq_94,freq_last59,freq_last60,freq_last61,freq_last62,freq_last63,freq_last64,freq_last65,freq_last66,freq_last67,freq_last68,freq_last69,freq_last70,freq_last71,freq_last72,freq_last73,freq_last74,freq_last75,freq_last76,freq_last77,freq_last78,freq_last79,freq_last80,freq_last81,freq_last82,freq_last83,freq_last84,freq_last85,freq_last86,freq_last87,freq_last88,freq_last89,freq_last90,freq_last91,freq_last92,freq_last93,freq_last94,freq_next59,freq_next60,freq_next61,freq_next62,freq_next63,freq_next64,freq_next65,freq_next66,freq_next67,freq_next68,freq_next69,freq_next70,freq_next71,freq_next72,freq_next73,freq_next74,freq_next75,freq_next76,freq_next77,freq_next78,freq_next79,freq_next80,freq_next81,freq_next82,freq_next83,freq_next84,freq_next85,freq_next86,freq_next87,freq_next88,freq_next89,freq_next90,freq_next91,freq_next92,freq_next93,freq_next94,years_data,year,month,day,hour,minute
0,2021-12-27 19:00:00,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021,12,27,19,0
1,2021-12-28 19:00:00,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [35]:
matrix = pd.DataFrame(matrix,columns = ['datetime','offered'] +add_columns +  [f'freq_{i}' for i in range(start_time+1,end_time+1) ]\
                      + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] + \
                      [f'freq_next{i}' for i in range(start_time+1,end_time+1) ]+ \
                       ['years_data']).sort_values('datetime').reset_index(drop = True)

In [37]:
matrix['year'] = matrix.datetime.map(lambda x:x.year)
matrix['month'] = matrix.datetime.map(lambda x:x.month)
matrix['day'] = matrix.datetime.map(lambda x:x.day)
matrix['hour'] = matrix.datetime.map(lambda x:x.hour)
matrix['minute'] = matrix.datetime.map(lambda x:x.minute)

In [39]:
matrix = matrix.sort_values('datetime').reset_index(drop = True)

In [40]:
read_data = matrix.iloc[:]

In [42]:
read_data = read_data[~read_data.offered.isna()].reset_index(drop = True)

In [45]:
read_data

,datetime,offered,actans_59,actans_60,actans_61,actans_62,actans_63,actans_64,actans_65,actans_66,actans_67,actans_68,actans_69,actans_70,actans_71,actans_72,actans_73,actans_74,actans_75,actans_76,actans_77,actans_78,actans_79,actans_80,actans_81,actans_82,actans_83,actans_84,actans_85,actans_86,actans_87,actans_88,actans_89,actans_90,actans_91,actans_92,actans_93,actans_94,actabn_59,actabn_60,actabn_61,actabn_62,actabn_63,actabn_64,actabn_65,actabn_66,actabn_67,actabn_68,actabn_69,actabn_70,actabn_71,actabn_72,actabn_73,actabn_74,actabn_75,actabn_76,actabn_77,actabn_78,actabn_79,actabn_80,actabn_81,actabn_82,actabn_83,actabn_84,actabn_85,actabn_86,actabn_87,actabn_88,actabn_89,actabn_90,actabn_91,actabn_92,actabn_93,actabn_94,absActHt_59,absActHt_60,absActHt_61,absActHt_62,absActHt_63,absActHt_64,absActHt_65,absActHt_66,absActHt_67,absActHt_68,absActHt_69,absActHt_70,absActHt_71,absActHt_72,absActHt_73,absActHt_74,absActHt_75,absActHt_76,absActHt_77,absActHt_78,absActHt_79,absActHt_80,absActHt_81,absActHt_82,absActHt_83,absActHt_84,absActHt_85,absActHt_86,absActHt_87,absActHt_88,absActHt_89,absActHt_90,absActHt_91,absActHt_92,absActHt_93,absActHt_94,absActSa_59,absActSa_60,absActSa_61,absActSa_62,absActSa_63,absActSa_64,absActSa_65,absActSa_66,absActSa_67,absActSa_68,absActSa_69,absActSa_70,absActSa_71,absActSa_72,absActSa_73,absActSa_74,absActSa_75,absActSa_76,absActSa_77,absActSa_78,absActSa_79,absActSa_80,absActSa_81,absActSa_82,absActSa_83,absActSa_84,absActSa_85,absActSa_86,absActSa_87,absActSa_88,absActSa_89,absActSa_90,absActSa_91,absActSa_92,absActSa_93,absActSa_94,freq_59,freq_60,freq_61,freq_62,freq_63,freq_64,freq_65,freq_66,freq_67,freq_68,freq_69,freq_70,freq_71,freq_72,freq_73,freq_74,freq_75,freq_76,freq_77,freq_78,freq_79,freq_80,freq_81,freq_82,freq_83,freq_84,freq_85,freq_86,freq_87,freq_88,freq_89,freq_90,freq_91,freq_92,freq_93,freq_94,freq_last59,freq_last60,freq_last61,freq_last62,freq_last63,freq_last64,freq_last65,freq_last66,freq_last67,freq_last68,freq_last69,freq_last70,freq_last71,freq_last72,freq_last73,freq_last74,freq_last75,freq_last76,freq_last77,freq_last78,freq_last79,freq_last80,freq_last81,freq_last82,freq_last83,freq_last84,freq_last85,freq_last86,freq_last87,freq_last88,freq_last89,freq_last90,freq_last91,freq_last92,freq_last93,freq_last94,freq_next59,freq_next60,freq_next61,freq_next62,freq_next63,freq_next64,freq_next65,freq_next66,freq_next67,freq_next68,freq_next69,freq_next70,freq_next71,freq_next72,freq_next73,freq_next74,freq_next75,freq_next76,freq_next77,freq_next78,freq_next79,freq_next80,freq_next81,freq_next82,freq_next83,freq_next84,freq_next85,freq_next86,freq_next87,freq_next88,freq_next89,freq_next90,freq_next91,freq_next92,freq_next93,freq_next94,years_data,year,month,day,hour,minute
0,2021-12-27 19:00:00,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021,12,27,19,0
1,2021-12-28 19:00:00,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [46]:
fill_na = read_data.iloc[:,1:].apply(lambda x:pd.to_numeric(x))

In [47]:
fill_na

,offered,actans_59,actans_60,actans_61,actans_62,actans_63,actans_64,actans_65,actans_66,actans_67,actans_68,actans_69,actans_70,actans_71,actans_72,actans_73,actans_74,actans_75,actans_76,actans_77,actans_78,actans_79,actans_80,actans_81,actans_82,actans_83,actans_84,actans_85,actans_86,actans_87,actans_88,actans_89,actans_90,actans_91,actans_92,actans_93,actans_94,actabn_59,actabn_60,actabn_61,actabn_62,actabn_63,actabn_64,actabn_65,actabn_66,actabn_67,actabn_68,actabn_69,actabn_70,actabn_71,actabn_72,actabn_73,actabn_74,actabn_75,actabn_76,actabn_77,actabn_78,actabn_79,actabn_80,actabn_81,actabn_82,actabn_83,actabn_84,actabn_85,actabn_86,actabn_87,actabn_88,actabn_89,actabn_90,actabn_91,actabn_92,actabn_93,actabn_94,absActHt_59,absActHt_60,absActHt_61,absActHt_62,absActHt_63,absActHt_64,absActHt_65,absActHt_66,absActHt_67,absActHt_68,absActHt_69,absActHt_70,absActHt_71,absActHt_72,absActHt_73,absActHt_74,absActHt_75,absActHt_76,absActHt_77,absActHt_78,absActHt_79,absActHt_80,absActHt_81,absActHt_82,absActHt_83,absActHt_84,absActHt_85,absActHt_86,absActHt_87,absActHt_88,absActHt_89,absActHt_90,absActHt_91,absActHt_92,absActHt_93,absActHt_94,absActSa_59,absActSa_60,absActSa_61,absActSa_62,absActSa_63,absActSa_64,absActSa_65,absActSa_66,absActSa_67,absActSa_68,absActSa_69,absActSa_70,absActSa_71,absActSa_72,absActSa_73,absActSa_74,absActSa_75,absActSa_76,absActSa_77,absActSa_78,absActSa_79,absActSa_80,absActSa_81,absActSa_82,absActSa_83,absActSa_84,absActSa_85,absActSa_86,absActSa_87,absActSa_88,absActSa_89,absActSa_90,absActSa_91,absActSa_92,absActSa_93,absActSa_94,freq_59,freq_60,freq_61,freq_62,freq_63,freq_64,freq_65,freq_66,freq_67,freq_68,freq_69,freq_70,freq_71,freq_72,freq_73,freq_74,freq_75,freq_76,freq_77,freq_78,freq_79,freq_80,freq_81,freq_82,freq_83,freq_84,freq_85,freq_86,freq_87,freq_88,freq_89,freq_90,freq_91,freq_92,freq_93,freq_94,freq_last59,freq_last60,freq_last61,freq_last62,freq_last63,freq_last64,freq_last65,freq_last66,freq_last67,freq_last68,freq_last69,freq_last70,freq_last71,freq_last72,freq_last73,freq_last74,freq_last75,freq_last76,freq_last77,freq_last78,freq_last79,freq_last80,freq_last81,freq_last82,freq_last83,freq_last84,freq_last85,freq_last86,freq_last87,freq_last88,freq_last89,freq_last90,freq_last91,freq_last92,freq_last93,freq_last94,freq_next59,freq_next60,freq_next61,freq_next62,freq_next63,freq_next64,freq_next65,freq_next66,freq_next67,freq_next68,freq_next69,freq_next70,freq_next71,freq_next72,freq_next73,freq_next74,freq_next75,freq_next76,freq_next77,freq_next78,freq_next79,freq_next80,freq_next81,freq_next82,freq_next83,freq_next84,freq_next85,freq_next86,freq_next87,freq_next88,freq_next89,freq_next90,freq_next91,freq_next92,freq_next93,freq_next94,years_data,year,month,day,hour,minute
0,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021,12,27,19,0
1,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [48]:
fill_na = fill_na.interpolate(method='linear')

fill_na['datetime'] = read_data.datetime
fill_na['date'] = fill_na.datetime.dt.date
fill_na = fill_na.dropna().reset_index(drop = True)

In [49]:
fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['month'],prefix = 'month').astype(int)],axis = 1)
fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['year'],prefix = 'year').astype(int)],axis = 1)
fill_na['week_of_year'] = fill_na['datetime'].dt.isocalendar().week
fill_na['weekday'] = fill_na['datetime'].dt.day_name()
fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['week_of_year'],prefix = 'week_of_year').astype(int)],axis = 1)
fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['weekday'],prefix = 'weekday').astype(int)],axis = 1)

In [50]:
fill_na

,offered,actans_59,actans_60,actans_61,actans_62,actans_63,actans_64,actans_65,actans_66,actans_67,actans_68,actans_69,actans_70,actans_71,actans_72,actans_73,actans_74,actans_75,actans_76,actans_77,actans_78,actans_79,actans_80,actans_81,actans_82,actans_83,actans_84,actans_85,actans_86,actans_87,actans_88,actans_89,actans_90,actans_91,actans_92,actans_93,actans_94,actabn_59,actabn_60,actabn_61,actabn_62,actabn_63,actabn_64,actabn_65,actabn_66,actabn_67,actabn_68,actabn_69,actabn_70,actabn_71,actabn_72,actabn_73,actabn_74,actabn_75,actabn_76,actabn_77,actabn_78,actabn_79,actabn_80,actabn_81,actabn_82,actabn_83,actabn_84,actabn_85,actabn_86,actabn_87,actabn_88,actabn_89,actabn_90,actabn_91,actabn_92,actabn_93,actabn_94,absActHt_59,absActHt_60,absActHt_61,absActHt_62,absActHt_63,absActHt_64,absActHt_65,absActHt_66,absActHt_67,absActHt_68,absActHt_69,absActHt_70,absActHt_71,absActHt_72,absActHt_73,absActHt_74,absActHt_75,absActHt_76,absActHt_77,absActHt_78,absActHt_79,absActHt_80,absActHt_81,absActHt_82,absActHt_83,absActHt_84,absActHt_85,absActHt_86,absActHt_87,absActHt_88,absActHt_89,absActHt_90,absActHt_91,absActHt_92,absActHt_93,absActHt_94,absActSa_59,absActSa_60,absActSa_61,absActSa_62,absActSa_63,absActSa_64,absActSa_65,absActSa_66,absActSa_67,absActSa_68,absActSa_69,absActSa_70,absActSa_71,absActSa_72,absActSa_73,absActSa_74,absActSa_75,absActSa_76,absActSa_77,absActSa_78,absActSa_79,absActSa_80,absActSa_81,absActSa_82,absActSa_83,absActSa_84,absActSa_85,absActSa_86,absActSa_87,absActSa_88,absActSa_89,absActSa_90,absActSa_91,absActSa_92,absActSa_93,absActSa_94,freq_59,freq_60,freq_61,freq_62,freq_63,freq_64,freq_65,freq_66,freq_67,freq_68,freq_69,freq_70,freq_71,freq_72,freq_73,freq_74,freq_75,freq_76,freq_77,freq_78,freq_79,freq_80,freq_81,freq_82,freq_83,freq_84,freq_85,freq_86,freq_87,freq_88,freq_89,freq_90,freq_91,freq_92,freq_93,freq_94,freq_last59,freq_last60,freq_last61,freq_last62,freq_last63,freq_last64,freq_last65,freq_last66,freq_last67,freq_last68,freq_last69,freq_last70,freq_last71,freq_last72,freq_last73,freq_last74,freq_last75,freq_last76,freq_last77,freq_last78,freq_last79,freq_last80,freq_last81,freq_last82,freq_last83,freq_last84,freq_last85,freq_last86,freq_last87,freq_last88,freq_last89,freq_last90,freq_last91,freq_last92,freq_last93,freq_last94,freq_next59,freq_next60,freq_next61,freq_next62,freq_next63,freq_next64,freq_next65,freq_next66,freq_next67,freq_next68,freq_next69,freq_next70,freq_next71,freq_next72,freq_next73,freq_next74,freq_next75,freq_next76,freq_next77,freq_next78,freq_next79,freq_next80,freq_next81,freq_next82,freq_next83,freq_next84,freq_next85,freq_next86,freq_next87,freq_next88,freq_next89,freq_next90,freq_next91,freq_next92,freq_next93,freq_next94,years_data,year,month,day,hour,minute,datetime,date,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,year_2023,year_2024,year_2025,week_of_year,weekday,week_of_year_1,week_of_year_2,week_of_year_3,week_of_year_4,week_of_year_5,week_of_year_6,week_of_year_7,week_of_year_8,week_of_year_9,week_of_year_10,week_of_year_11,week_of_year_12,week_of_year_13,week_of_year_14,week_of_year_15,week_of_year_16,week_of_year_17,week_of_year_18,week_of_year_19,week_of_year_20,week_of_year_21,week_of_year_22,week_of_year_23,week_of_year_24,week_of_year_25,week_of_year_26,week_of_year_27,week_of_year_28,week_of_year_29,week_of_year_30,week_of_year_31,week_of_year_32,week_of_year_33,week_of_year_34,week_of_year_35,week_of_year_36,week_of_year_37,week_of_year_38,week_of_year_39,week_of_year_40,week_of_year_41,week_of_year_42,week_of_year_43,week_of_year_44,week_of_year_45,week_of_year_46,week_of_year_47,week_of_year_48,week_of_year_49,week_of_year_50,week_of_year_51,week_of_year_52,weekday_Friday,weekday_Monday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday
0,28.0,3.0,4.0,2962.285714,13.000000,8.0,4.0,11224.571429,15312.800000,10.0,4.0,17510.000000,3048.285714,13.0,1.0,39874.250000,7592.742857,12.0,6.0,7

In [51]:
matrix_predict = []

# 遍历每个时间点
for time,time1,time2 in zip(unique_times,np.roll(unique_times, shift=-1),np.roll(unique_times, shift=1)):
    # 过滤出当前时间点的数据
    time_data = data[data['time'] == time].set_index('date')['offered']    
    full_dates = pd.date_range(start=unique_dates.min(), end=unique_dates.max()+ pd.offsets.BDay(start_time), freq='B')  # 仅工作日
    full_dates = full_dates.difference(holiday_datetime)
    time_data = time_data.groupby(time_data.index).sum()    
    time_data = time_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
    shifted_data = pd.concat([time_data.shift(i) for i in range(start_time,end_time)], axis=1)
    
    
    add_data = data[data['time'] == time].set_index('date')[add_column]
    add_data = add_data.groupby(add_data.index).sum()
    add_data = add_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
    add_data_ = pd.concat([add_data.shift(i) for i in range(start_time,end_time)], axis=1)
    
    
    
    time_data1 = data[data['time'] == time1].set_index('date')['offered'] 
    time_data1 = time_data1.groupby(time_data1.index).sum()
    time_data1 = time_data1.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
    shifted_data1 = pd.concat([time_data1.shift(i) for i in range(start_time,end_time)], axis=1)
    
    time_data2 = data[data['time'] == time2].set_index('date')['offered']  
    time_data2 = time_data2.groupby(time_data2.index).sum()
    time_data2 = time_data2.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
    shifted_data2 = pd.concat([time_data2.shift(i) for i in range(start_time,end_time)], axis=1)
    
    
    year_data = time_data.shift(251)
    

    # 将当前时间点、前半小时和后半小时的数据拼接
    combined_data = pd.concat([time_data,add_data_, shifted_data,shifted_data1,shifted_data2,year_data], axis=1)
    
    # 创建日期+时间列
    datetime_column = np.array([pd.Timestamp(date) + pd.Timedelta(hours=time.hour, minutes=time.minute) for date in full_dates]).reshape(-1, 1)
    
    # 将日期+时间列添加到数据中
    combined_data_with_datetime = np.hstack([datetime_column,combined_data.to_numpy()])[-start_time:]
    
    
    
    # 将结果存入矩阵
    matrix_predict.append(combined_data_with_datetime)

max_rows = max(arr.shape[0] for arr in matrix_predict)
# 将每个时间点的数据填充到最大行数
for i in range(len(matrix_predict)):
    if matrix_predict[i].shape[0] < max_rows:
        padding = np.full((max_rows - matrix_predict[i].shape[0], 11), np.nan)  # 用NaN填充（10列数据 + 1列时间）
        matrix_predict[i] = np.vstack([matrix_predict[i], padding])

# 将矩阵堆叠成一个大的二维数组
matrix_predict = np.vstack(matrix_predict)

matrix_predict = pd.DataFrame(matrix_predict,columns = ['datetime','offered'] + add_columns+ [f'freq_{i}' for i in range(start_time+1,end_time+1) ]\
                      + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] + [f'freq_next{i}' for i in range(start_time+1,end_time+1) ]+ \
                       ['years_data']).sort_values('datetime').reset_index(drop = True)

matrix_predict['year'] = matrix_predict.datetime.map(lambda x:x.year)
matrix_predict['month'] = matrix_predict.datetime.map(lambda x:x.month)
matrix_predict['day'] = matrix_predict.datetime.map(lambda x:x.day)
matrix_predict['hour'] = matrix_predict.datetime.map(lambda x:x.hour)
matrix_predict['minute'] = matrix_predict.datetime.map(lambda x:x.minute)

matrix_predict = matrix_predict.sort_values('datetime').reset_index(drop = True)

read_data_predict = matrix_predict.iloc[:]


In [52]:
fill_na_predict = read_data_predict.iloc[:,2:].apply(lambda x:pd.to_numeric(x))

fill_na_predict = fill_na_predict.interpolate(method='linear')
fill_na_predict = fill_na_predict.fillna(1)

fill_na_predict['datetime'] = read_data_predict.datetime
fill_na_predict['date'] = fill_na_predict.datetime.dt.date

fill_na_predict = pd.concat([fill_na_predict,pd.get_dummies(fill_na_predict['month'],prefix = 'month').astype(int)],axis = 1)
fill_na_predict = pd.concat([fill_na_predict,pd.get_dummies(fill_na_predict['year'], prefix = 'year').astype(int)],axis = 1)
fill_na_predict['week_of_year'] = fill_na_predict['datetime'].dt.isocalendar().week
fill_na_predict['weekday'] = fill_na_predict['datetime'].dt.day_name()
fill_na_predict = pd.concat([fill_na_predict,pd.get_dummies(fill_na_predict['week_of_year'],prefix = 'week_of_year').astype(int)],axis = 1)
fill_na_predict = pd.concat([fill_na_predict,pd.get_dummies(fill_na_predict['weekday'], prefix = 'weekday').astype(int)],axis = 1)

In [53]:
fill_na_predict

,actans_59,actans_60,actans_61,actans_62,actans_63,actans_64,actans_65,actans_66,actans_67,actans_68,actans_69,actans_70,actans_71,actans_72,actans_73,actans_74,actans_75,actans_76,actans_77,actans_78,actans_79,actans_80,actans_81,actans_82,actans_83,actans_84,actans_85,actans_86,actans_87,actans_88,actans_89,actans_90,actans_91,actans_92,actans_93,actans_94,actabn_59,actabn_60,actabn_61,actabn_62,actabn_63,actabn_64,actabn_65,actabn_66,actabn_67,actabn_68,actabn_69,actabn_70,actabn_71,actabn_72,actabn_73,actabn_74,actabn_75,actabn_76,actabn_77,actabn_78,actabn_79,actabn_80,actabn_81,actabn_82,actabn_83,actabn_84,actabn_85,actabn_86,actabn_87,actabn_88,actabn_89,actabn_90,actabn_91,actabn_92,actabn_93,actabn_94,absActHt_59,absActHt_60,absActHt_61,absActHt_62,absActHt_63,absActHt_64,absActHt_65,absActHt_66,absActHt_67,absActHt_68,absActHt_69,absActHt_70,absActHt_71,absActHt_72,absActHt_73,absActHt_74,absActHt_75,absActHt_76,absActHt_77,absActHt_78,absActHt_79,absActHt_80,absActHt_81,absActHt_82,absActHt_83,absActHt_84,absActHt_85,absActHt_86,absActHt_87,absActHt_88,absActHt_89,absActHt_90,absActHt_91,absActHt_92,absActHt_93,absActHt_94,absActSa_59,absActSa_60,absActSa_61,absActSa_62,absActSa_63,absActSa_64,absActSa_65,absActSa_66,absActSa_67,absActSa_68,absActSa_69,absActSa_70,absActSa_71,absActSa_72,absActSa_73,absActSa_74,absActSa_75,absActSa_76,absActSa_77,absActSa_78,absActSa_79,absActSa_80,absActSa_81,absActSa_82,absActSa_83,absActSa_84,absActSa_85,absActSa_86,absActSa_87,absActSa_88,absActSa_89,absActSa_90,absActSa_91,absActSa_92,absActSa_93,absActSa_94,freq_59,freq_60,freq_61,freq_62,freq_63,freq_64,freq_65,freq_66,freq_67,freq_68,freq_69,freq_70,freq_71,freq_72,freq_73,freq_74,freq_75,freq_76,freq_77,freq_78,freq_79,freq_80,freq_81,freq_82,freq_83,freq_84,freq_85,freq_86,freq_87,freq_88,freq_89,freq_90,freq_91,freq_92,freq_93,freq_94,freq_last59,freq_last60,freq_last61,freq_last62,freq_last63,freq_last64,freq_last65,freq_last66,freq_last67,freq_last68,freq_last69,freq_last70,freq_last71,freq_last72,freq_last73,freq_last74,freq_last75,freq_last76,freq_last77,freq_last78,freq_last79,freq_last80,freq_last81,freq_last82,freq_last83,freq_last84,freq_last85,freq_last86,freq_last87,freq_last88,freq_last89,freq_last90,freq_last91,freq_last92,freq_last93,freq_last94,freq_next59,freq_next60,freq_next61,freq_next62,freq_next63,freq_next64,freq_next65,freq_next66,freq_next67,freq_next68,freq_next69,freq_next70,freq_next71,freq_next72,freq_next73,freq_next74,freq_next75,freq_next76,freq_next77,freq_next78,freq_next79,freq_next80,freq_next81,freq_next82,freq_next83,freq_next84,freq_next85,freq_next86,freq_next87,freq_next88,freq_next89,freq_next90,freq_next91,freq_next92,freq_next93,freq_next94,years_data,year,month,day,hour,minute,datetime,date,month_1,month_2,month_3,month_4,year_2025,week_of_year,weekday,week_of_year_3,week_of_year_4,week_of_year_5,week_of_year_6,week_of_year_7,week_of_year_8,week_of_year_9,week_of_year_10,week_of_year_11,week_of_year_12,week_of_year_13,week_of_year_14,weekday_Friday,weekday_Monday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday
0,1.0,1.0,1.000,1.0,2.0,2.0,1080.139,22.0,2.0,1.0,83.646,15.0,0.0,2.0,1.000,1.0,0.0,1.0,1.000,1.0,1.0,3.0,7686.174,7.0,2.0,0.0,6008.325,387.0,1.0,0.0,463.193,14.0,1.0,1.0,1.000,1.0,1.0,2.0,1138.482,1.0,0.0,1.0,1.0000,1.0,0.0,2.0,1.000,1.0,1.0,2.0,808.410,264.0,0.0,3.0,1.000,1.0,0.0,1.0,1.000,1.0,1.0,1.0,1.000,1.0,0.0,1.0,1.000,1.0,1.0,1.0,305.792,14.0,1.0,1.0,1.000,1.0,1.0,1.0,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.0,1.0,1.00000,1.000000,1.0,1.0,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.0,4.0,2.0,2.0,1.0,4.0,2.0,1.0,1.0,2.0,1.0,2.0,3.0,3.0,1.0,1.0,1.0,2

In [55]:
fill_na.shape

(494, 335)

In [56]:
fill_na_predict.shape

(1334, 284)

In [57]:
X_columns = [f'freq_{i}' for i in range(start_time+1,end_time+1) ] + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] +\
            [f'freq_next{i}' for i in range(start_time+1,end_time+1) ] + ['years_data'] + [f'month_{i}' for i in range(1,13)] +\
            [f'year_{i}' for i in np.unique(fill_na.year)] +\
            [f'week_of_year_{i}' for i in fill_na.week_of_year.drop_duplicates().tolist()] +\
                add_columns
            
X_columns2 = [f'freq_{i}' for i in range(start_time+1,end_time+1) ] + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] +\
            [f'freq_next{i}' for i in range(start_time+1,end_time+1) ] + ['years_data'] + [f'month_{i}' for i in range(1,13)] +\
            [f'year_{i}' for i in np.unique(fill_na.year)] +\
            [f'week_of_year_{i}' for i in fill_na.week_of_year.drop_duplicates().tolist()] +\
                add_columns


lstm_data = fill_na.dropna(axis = 0)

In [66]:
len(X_columns2)

320

In [67]:
for i in X_columns:
    try:
        fill_na_predict[i]
    except:
        fill_na_predict[i] = 0

In [68]:
fill_na_predict

,actans_59,actans_60,actans_61,actans_62,actans_63,actans_64,actans_65,actans_66,actans_67,actans_68,actans_69,actans_70,actans_71,actans_72,actans_73,actans_74,actans_75,actans_76,actans_77,actans_78,actans_79,actans_80,actans_81,actans_82,actans_83,actans_84,actans_85,actans_86,actans_87,actans_88,actans_89,actans_90,actans_91,actans_92,actans_93,actans_94,actabn_59,actabn_60,actabn_61,actabn_62,actabn_63,actabn_64,actabn_65,actabn_66,actabn_67,actabn_68,actabn_69,actabn_70,actabn_71,actabn_72,actabn_73,actabn_74,actabn_75,actabn_76,actabn_77,actabn_78,actabn_79,actabn_80,actabn_81,actabn_82,actabn_83,actabn_84,actabn_85,actabn_86,actabn_87,actabn_88,actabn_89,actabn_90,actabn_91,actabn_92,actabn_93,actabn_94,absActHt_59,absActHt_60,absActHt_61,absActHt_62,absActHt_63,absActHt_64,absActHt_65,absActHt_66,absActHt_67,absActHt_68,absActHt_69,absActHt_70,absActHt_71,absActHt_72,absActHt_73,absActHt_74,absActHt_75,absActHt_76,absActHt_77,absActHt_78,absActHt_79,absActHt_80,absActHt_81,absActHt_82,absActHt_83,absActHt_84,absActHt_85,absActHt_86,absActHt_87,absActHt_88,absActHt_89,absActHt_90,absActHt_91,absActHt_92,absActHt_93,absActHt_94,absActSa_59,absActSa_60,absActSa_61,absActSa_62,absActSa_63,absActSa_64,absActSa_65,absActSa_66,absActSa_67,absActSa_68,absActSa_69,absActSa_70,absActSa_71,absActSa_72,absActSa_73,absActSa_74,absActSa_75,absActSa_76,absActSa_77,absActSa_78,absActSa_79,absActSa_80,absActSa_81,absActSa_82,absActSa_83,absActSa_84,absActSa_85,absActSa_86,absActSa_87,absActSa_88,absActSa_89,absActSa_90,absActSa_91,absActSa_92,absActSa_93,absActSa_94,freq_59,freq_60,freq_61,freq_62,freq_63,freq_64,freq_65,freq_66,freq_67,freq_68,freq_69,freq_70,freq_71,freq_72,freq_73,freq_74,freq_75,freq_76,freq_77,freq_78,freq_79,freq_80,freq_81,freq_82,freq_83,freq_84,freq_85,freq_86,freq_87,freq_88,freq_89,freq_90,freq_91,freq_92,freq_93,freq_94,freq_last59,freq_last60,freq_last61,freq_last62,freq_last63,freq_last64,freq_last65,freq_last66,freq_last67,freq_last68,freq_last69,freq_last70,freq_last71,freq_last72,freq_last73,freq_last74,freq_last75,freq_last76,freq_last77,freq_last78,freq_last79,freq_last80,freq_last81,freq_last82,freq_last83,freq_last84,freq_last85,freq_last86,freq_last87,freq_last88,freq_last89,freq_last90,freq_last91,freq_last92,freq_last93,freq_last94,freq_next59,freq_next60,freq_next61,freq_next62,freq_next63,freq_next64,freq_next65,freq_next66,freq_next67,freq_next68,freq_next69,freq_next70,freq_next71,freq_next72,freq_next73,freq_next74,freq_next75,freq_next76,freq_next77,freq_next78,freq_next79,freq_next80,freq_next81,freq_next82,freq_next83,freq_next84,freq_next85,freq_next86,freq_next87,freq_next88,freq_next89,freq_next90,freq_next91,freq_next92,freq_next93,freq_next94,years_data,year,month,day,hour,minute,datetime,date,month_1,month_2,month_3,month_4,year_2025,week_of_year,weekday,week_of_year_3,week_of_year_4,week_of_year_5,week_of_year_6,week_of_year_7,week_of_year_8,week_of_year_9,week_of_year_10,week_of_year_11,week_of_year_12,week_of_year_13,week_of_year_14,weekday_Friday,weekday_Monday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,year_2023,year_2024,week_of_year_15,week_of_year_16,week_of_year_17,week_of_year_18,week_of_year_19,week_of_year_20,week_of_year_21,week_of_year_22,week_of_year_23,week_of_year_24,week_of_year_25,week_of_year_26,week_of_year_27,week_of_year_28,week_of_year_29,week_of_year_30,week_of_year_31,week_of_year_32,week_of_year_33,week_of_year_34,week_of_year_35,week_of_year_36,week_of_year_37,week_of_year_38,week_of_year_39,week_of_year_40,week_of_year_41,week_of_year_42,week_of_year_43,week_of_year_44,week_of_year_45,week_of_year_46,week_of_year_47,week_of_year_48,week_of_year_49,week_of_year_50,week_of_year_51,week_of_year_52,week_of_year_1,week_of_year_2
0,1.0,1.0,1.000,1.0,2.0,2.0,1080.139,22.0,2.0,1.0,83.646,15.0,0.0,2.0,1.000,1.0,0.0,1.0,1.000,1.0,1.0,3.0,7686.174,7.0,2.0,0.0,6008.325,387.0,1.0,0.0,463.193

In [69]:
lstm_data = lstm_data[lstm_data.year>=2021].reset_index(drop = True)

In [70]:
week_dict = {'Friday':5, 'Monday':1, 'Tuesday':2, 'Wednesday':3, 'Thursday':4}

In [71]:
lstm_data['week_num'] = lstm_data['weekday'].map(lambda x:week_dict[x])
fill_na_predict['week_num'] = fill_na_predict['weekday'].map(lambda x:week_dict[x])

In [73]:
import numpy as np

def calculate_mape(actual, predicted):
    """
    计算 MAPE（平均绝对百分比误差）

    参数:
    
    
    actual (array-like): 实际值数组。
    predicted (array-like): 预测值数组。

    返回:
    float: MAPE 值。
    """
    # 将输入转换为 numpy 数组
    actual = np.array(actual)
    predicted = np.array(predicted)

    # 避免除以零的情况
    if np.any(actual == 0):
        raise ValueError("实际值中包含零，无法计算 MAPE。")

    # 计算 MAPE
    mape = np.mean(np.abs((actual - predicted) / actual)) * 100
    return mape


In [72]:
all_predict = pd.DataFrame()
all_feature = pd.DataFrame()

In [ ]:

all_predict = pd.DataFrame()
all_feature = pd.DataFrame()
#for week_ in [1,2,3,4,5]:
for (hour,minute) in lstm_data[['hour','minute']].drop_duplicates().sort_values(['hour','minute']).to_numpy()[:]:
    now_lstm_data = lstm_data[(lstm_data.hour == hour)&(lstm_data.minute == minute)]
    #now_lstm_data = now_lstm_data[now_lstm_data.week_num == week_]
    now_fill_na_predict = fill_na_predict[(fill_na_predict.hour == hour)&(fill_na_predict.minute == minute)]
    #now_fill_na_predict = now_fill_na_predict[now_fill_na_predict.week_num == week_]

    now_time_list = now_lstm_data.datetime.to_numpy()
    now_lstm_data = now_lstm_data[['offered']+['datetime'] +X_columns].reset_index(drop = True)
    # 归一化数据

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(now_lstm_data[['offered']][:-start_time])
    # 设置时间步长
    time_step = start_time
    # 创建训练数据集
    X_train_1, y_train =now_lstm_data[X_columns][:-start_time] , scaled_data

    X_test_1, y_test =now_lstm_data[X_columns][-start_time:] , now_lstm_data[['offered']][-start_time:]

    X_train_1 = np.array(X_train_1)
    y_train = np.array(y_train)
    X_test_1 = np.array(X_test_1)
    y_test = np.array(y_test)
    X_test_1 = X_test_1.reshape(X_test_1.shape[0], X_test_1.shape[1], 1)

    # 重塑输入数据为 [samples, time steps, features] 格式
    X_train_1 = X_train_1.reshape(X_train_1.shape[0], X_train_1.shape[1], 1)
    # 构建LSTM模型
    model_1 = Sequential()
    model_1.add(LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
    model_1.add(LSTM(50, return_sequences=False))
    model_1.add(Dense(25))
    model_1.add(Dense(1))

    # 编译模型
    model_1.compile(optimizer='adam', loss='mean_squared_error')
    model_1.fit(X_train_1, y_train, batch_size=32, epochs=20)
    train_predict_1 = model_1.predict(X_train_1)
    train_predict_1 = scaler.inverse_transform(train_predict_1)
    True_data_1 = scaler.inverse_transform(y_train.reshape(-1, 1))
    print(calculate_mape(train_predict_1, True_data_1))
    return_y_1 =  model_1.predict(X_test_1)
    return_y_pred_1 = scaler.inverse_transform(np.array(return_y_1).reshape(-1, 1))
    now_predict_1 = pd.DataFrame(return_y_pred_1,columns = ['predict'])
    now_predict_1['datetime'] = now_time_list[-start_time:]


    now_lstm_data = now_lstm_data[['offered'] +X_columns2].reset_index(drop = True)

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(now_lstm_data[['offered']][:-start_time])
    # 设置时间步长
    time_step = start_time
    # 创建训练数据集

    X_train_2, y_train =now_lstm_data[X_columns2][:-start_time] , scaled_data

    X_test_2, y_test =now_lstm_data[X_columns2][-start_time:] , now_lstm_data[['offered']][-start_time:]

    X_train_2 = np.array(X_train_2)
    y_train = np.array(y_train)
    X_test_2 = np.array(X_test_2)
    y_test = np.array(y_test)
    X_test_2 = X_test_2.reshape(X_test_2.shape[0], X_test_2.shape[1], 1)

    # 重塑输入数据为 [samples, time steps, features] 格式
    X_train_2 = X_train_2.reshape(X_train_2.shape[0], X_train_2.shape[1], 1)
    # 构建LSTM模型
    model_2 = Sequential()
    model_2.add(LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
    model_2.add(LSTM(50, return_sequences=False))
    model_2.add(Dense(25))
    model_2.add(Dense(1))

    # 编译模型
    model_2.compile(optimizer='adam', loss='mean_squared_error')
    model_2.fit(X_train_2, y_train, batch_size=32, epochs=20)
    train_predict_2 = model_2.predict(X_train_2)
    train_predict_2 = scaler.inverse_transform(train_predict_2)
    True_data_2 = scaler.inverse_transform(y_train.reshape(-1, 1))
    print(calculate_mape(train_predict_2, True_data_2))
    return_y_2 =  model_2.predict(X_test_2)
    return_y_pred_2 = scaler.inverse_transform(np.array(return_y_2).reshape(-1, 1))
    now_predict_2 = pd.DataFrame(return_y_pred_2,columns = ['predict'])
    now_predict_2['datetime'] = now_time_list[-start_time:]
    print(calculate_mape(y_test, return_y_pred_2))
    if calculate_mape(y_test, return_y_pred_1) < calculate_mape(y_test, return_y_pred_2):
        all_predict = pd.concat([now_predict_1,all_predict])
        future_X = now_fill_na_predict[X_columns].to_numpy()
        future_data = scaler.inverse_transform(np.array(\
                        model_1.predict(future_X.reshape(future_X.shape[0], future_X.shape[1], 1))).reshape(-1, 1))
        future_data = pd.DataFrame(future_data,columns = ['predict'])
        all_feature = pd.concat([all_feature,future_data])
    else:
        all_predict = pd.concat([now_predict_2,all_predict])
        future_X = now_fill_na_predict[X_columns2].to_numpy()
        future_data = scaler.inverse_transform(np.array(\
                        model_2.predict(future_X.reshape(future_X.shape[0], future_X.shape[1], 1))).reshape(-1, 1))
        future_data = pd.DataFrame(future_data,columns = ['predict'])
        all_feature = pd.concat([all_feature,future_data])
all_predict = pd.merge(all_predict,lstm_data[['offered','datetime']],on = ['datetime'])
all_predict = all_predict.sort_values('datetime').reset_index(drop = True)
all_feature.to_excel(r'return2.xlsx')

In [78]:
hour=19
minute=0
now_lstm_data = lstm_data[(lstm_data.hour == hour)&(lstm_data.minute == minute)]
#now_lstm_data = now_lstm_data[now_lstm_data.week_num == week_]
now_fill_na_predict = fill_na_predict[(fill_na_predict.hour == hour)&(fill_na_predict.minute == minute)]
#now_fill_na_predict = now_fill_na_predict[now_fill_na_predict.week_num == week_]
now_time_list = now_lstm_data.datetime.to_numpy()
now_lstm_data = now_lstm_data[['offered']+['datetime'] +X_columns].reset_index(drop = True)
# 归一化数据


In [82]:
# Normalization 

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(now_lstm_data[['offered']][:-start_time])
# set time step
time_step = start_time
 

In [85]:
# Train dataset
X_train_1, y_train =now_lstm_data[X_columns][:-start_time] , scaled_data

X_test_1, y_test =now_lstm_data[X_columns][-start_time:] , now_lstm_data[['offered']][-start_time:]


In [90]:
scaled_data

array([[0.62790698],
       [0.62790698],
       [0.44186047],
       [0.51162791],
       [0.3255814 ],
       [0.41860465],
       [0.37209302],
       [0.39534884],
       [0.34883721],
       [0.27906977],
       [0.25581395],
       [0.41860465],
       [0.34883721],
       [0.27906977],
       [0.30232558],
       [0.09302326],
       [0.13953488],
       [0.37209302],
       [0.09302326],
       [0.34883721],
       [0.11627907],
       [0.3255814 ],
       [0.23255814],
       [0.30232558],
       [0.37209302],
       [0.30232558],
       [0.1627907 ],
       [0.23255814],
       [0.23255814],
       [0.23255814],
       [0.23255814],
       [0.27906977],
       [0.25581395],
       [0.34883721],
       [0.55813953],
       [0.37209302],
       [0.30232558],
       [0.27906977],
       [0.27906977],
       [0.30232558],
       [0.20930233],
       [0.46511628],
       [0.51162791],
       [0.34883721],
       [0.1627907 ],
       [0.23255814],
       [0.34883721],
       [0.209

In [88]:
X_test_1.shape

(58, 320)

In [ ]:
now_time_list = now_lstm_data.datetime.to_numpy()
now_lstm_data = now_lstm_data[['offered']+['datetime'] +X_columns].reset_index(drop = True)
# 归一化数据

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(now_lstm_data[['offered']][:-start_time])
# 设置时间步长
time_step = start_time
# 创建训练数据集
X_train_1, y_train =now_lstm_data[X_columns][:-start_time] , scaled_data

X_test_1, y_test =now_lstm_data[X_columns][-start_time:] , now_lstm_data[['offered']][-start_time:]

X_train_1 = np.array(X_train_1)
y_train = np.array(y_train)
X_test_1 = np.array(X_test_1)
y_test = np.array(y_test)
X_test_1 = X_test_1.reshape(X_test_1.shape[0], X_test_1.shape[1], 1)

# 重塑输入数据为 [samples, time steps, features] 格式
X_train_1 = X_train_1.reshape(X_train_1.shape[0], X_train_1.shape[1], 1)

In [ ]:
all_predict['diff'] = (all_predict.predict - all_predict.offered)

In [ ]:
all_predict['hour'] = all_predict.datetime.dt.hour

In [ ]:
pd.merge(all_predict,lstm_data[['offered','datetime']],on = ['datetime'])

In [ ]:
calculate_mape(all_predict.offered,all_predict.predict)

In [ ]:
all_predict.to_excel(r'return1-.xlsx')

In [ ]:
data[add_column + ['offered']].corr()

In [ ]:
all_feature

In [ ]:
all_predict = pd.concat([now_predict_1,all_predict])
future_X = now_fill_na_predict[X_columns].to_numpy()
future_data = scaler.inverse_transform(np.array(\
                model_1.predict(future_X.reshape(future_X.shape[0], future_X.shape[1], 1))).reshape(-1, 1))
future_data = pd.DataFrame(future_data,columns = ['predict'])
all_feature = pd.concat([all_feature,future_data])

In [ ]:
all_predict

In [ ]:
all_predict = pd.merge(all_predict,lstm_data[['offered','datetime']],on = ['datetime'])

In [ ]:
all_predict.sort_values('datetime')

In [ ]:
all_feature = all_feature[:-58]

In [ ]:
all_feature

In [ ]:
time_ = []
for (hour,minute) in fill_na_predict[['hour','minute']].drop_duplicates().to_numpy()[:]:
    now_fill_na_predict = fill_na_predict[(fill_na_predict.hour == hour)&(fill_na_predict.minute == minute)]
    time_.append(now_fill_na_predict.datetime.to_numpy())
all_feature['datetime'] =  np.hstack(time_)
all_feature.predict = all_feature.predict.map(lambda x:max(0,x))
all_feature.to_excel(r'feature2.xlsx')

In [ ]:
all_predict

In [ ]:
all_feature

In [ ]:
calculate_mape(all_predict.predict, all_predict.offered)

In [ ]:
(np.mean((all_predict.predict-all_predict.offered)**2))**0.5

In [ ]:
all_predict['hour'] = all_predict.datetime.dt.hour

In [ ]:
for hour in np.unique(all_predict.hour):
    _ = all_predict[all_predict.hour == hour]
    print(f'hour is {hour} the mape ：',calculate_mape(_.predict, _.offered))

In [ ]:
for hour in np.unique(all_predict.hour):
    _ = all_predict[all_predict.hour == hour]
    print(f'hour is {hour} the mse ：',(np.mean((_.predict-_.offered)**2))**0.5)

In [ ]:
1

In [ ]:
_ = all_predict[all_predict.hour == 11]

In [ ]:
calculate_mape(_.predict, _.offered)

In [ ]:
all_predict

In [ ]:
feature_[['predict','datetime']]

In [ ]:
all_predict = pd.merge(all_predict,lstm_data[['offered','datetime']],on = ['datetime'])
all_predict = all_predict.sort_values('datetime').reset_index(drop = True)


In [ ]:
all_predict.to_excel(r'return.xlsx')

In [ ]:
datetime

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
data11 = lstm_data[lstm_data.hour == 11]

In [ ]:
data11.groupby('datetime').min().offered.sort_values()

In [ ]:

import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt

# 创建示例数据
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=all_predict['datetime'],  # 将 '日期' 列作为 X 轴
    y=all_predict['offered'],  # 将 '销售额' 列作为 Y 轴
    mode='lines+markers',  # 折线 + 数据点
    name='真实值'  # 图例名称
))
fig.add_trace(go.Scatter(
    x=all_predict['datetime'],  # 将 '日期' 列作为 X 轴
    y=all_predict['predict'],  # 将 '销售额' 列作为 Y 轴
    mode='lines+markers',  # 折线 + 数据点
    name='预测值'  # 图例名称
))

# 设置坐标轴名称
fig.update_layout(
    title='折线图示例',
    xaxis_title='X轴',
    yaxis_title='Y轴',
    template='plotly_white'
)
fig.show()

In [ ]:

# 可视化训练数据的预测结果
plt.figure(figsize=(10, 6))
plt.plot( all_predict['datetime'], all_predict['offered'] , label='Actual Data')
plt.plot( all_predict['datetime'], all_predict['predict'], label='Predicted Data')
plt.legend()
plt.show()
# 预测未来数据
future_steps = 30
future_predictions = []

last_sequence = X_train[-1]


In [ ]:
now_lstm_data

In [ ]:
now_lstm_data.offered.min()

In [ ]:
_ = lstm_data.groupby(['year','month','day']).offered.sum()
_.sort_values()[:30]

In [ ]:
now_lstm_data[X_columns]

In [ ]:

# 可视化训练数据的预测结果
plt.figure(figsize=(10, 6))
plt.plot(now_lstm_data[['offered']][1000:].to_numpy(), label='Actual Data')
plt.plot(return_y_pred, label='Predicted Data')
plt.legend()
plt.show()
# 预测未来数据
future_steps = 30
future_predictions = []

last_sequence = X_train[-1]


In [ ]:
predict_y =  model.predict( now_lstm_data[X_columns][:1000].to_numpy().reshape(-1, X_train.shape[1], 1))
predict_y = scaler.inverse_transform(np.array(predict_y).reshape(-1, 1))

In [ ]:
calculate_mape(now_lstm_data[['offered']][:1000].to_numpy(), predict_y)

In [ ]:
_ = ((np.abs((now_lstm_data[['offered']][1000:].to_numpy() - return_y_pred)/now_lstm_data[['offered']][1000:].to_numpy())))*100

In [ ]:
_.sort()

In [ ]:
np.argmax(_)

In [ ]:
_[298]

In [ ]:
now_lstm_data[['offered']][1000:].to_numpy()[298]

In [ ]:
import pandas as pd
import numpy as np
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from keras.optimizers import Adam
from tensorflow.keras.layers import Dense, LSTM
from keras.layers import  Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import datetime
import numpy as np
from datetime import time


data = pd.read_excel('workspace/RS_KSR_WFM_STAT_2025_05_14.xlsx')
#data = pd.read_excel('RS_KSR_WFM_STAT_2025_05_14.xlsx')

pd.set_option('display.max_rows', 100)

data.columns = ['conversation_start_interval_tmst', 'Time', 'offered', 'actans',
       'actabn', 'absActHt', 'absActSa', 'parent', 'child', 'fiscalDate',
       'fiscalYear', 'ficalQuarter', 'fiscalMonth', 'fiscalWeek', 'DOW']

data = data[~data .offered.isna()]
data = data[data .offered != 0]
data_=data
month_number='2025-03-15'
data = data[data.conversation_start_interval_tmst<=pd.to_datetime('{} 00:00:00'.format(month_number))]
# 假设数据有两列：'timestamp' 和 'value'
holiday_data = pd.read_excel(r'workspace/holiday.xlsx')
#holiday_data = pd.read_excel(r'holiday.xlsx')

holiday_data['~is_holiday'] = 0
Holiday_name = ['Christmas Day', 'Columbus Day',
       'Independence Day', 'Labor Day', 'Martin Luther King Jr. Day',
       'Memorial Day', "New Year's Day", "Presidents' Day", 'Thanksgiving Day',
       'Veterans Day']
holiday_data['date'] = holiday_data.Date.dt.date
data['datetime'] = pd.to_datetime(data.conversation_start_interval_tmst)
data['date'] = data.datetime.dt.date

data.datetime = data.datetime.dt.floor('T')
data['datetime'] = data['datetime'].apply(
    lambda x: x.ceil('30T') if x.minute == 29 else x
)
data['datetime'] = data['datetime'].apply(
    lambda x: x.ceil('H') if x.minute == 59 else x
)
data['time'] = data.datetime.dt.time
data = data.sort_values('datetime').reset_index(drop = True)
data = data.sort_values(by='datetime')
data = data[data.fiscalDate<pd.to_datetime('2025-04-15 00:00:00')]

# 获取所有唯一的日期
unique_dates = data['date'].unique()
data = pd.merge(data,holiday_data,on = 'date',how = 'left').fillna(1)
data = data[data['~is_holiday'] == 1]

holiday_datetime = holiday_data.date.to_numpy()


unique_times = [
time(9, 0), time(9, 30), time(10, 0),
time(10, 30), time(11, 0), time(11, 30),
time(12, 0), time(12, 30), time(13, 0),
time(13, 30), time(14, 0), time(14, 30),
time(15, 0), time(15, 30), time(16, 0),
time(16, 30), time(17, 0), time(17, 30),
time(18, 0), time(18, 30), time(19, 0)]


def calculate_mape(actual, predicted):
    """
    计算 MAPE（平均绝对百分比误差）

    参数:
    
    
    actual (array-like): 实际值数组。
    predicted (array-like): 预测值数组。

    返回:
    float: MAPE 值。
    """
    # 将输入转换为 numpy 数组
    actual = np.array(actual)
    predicted = np.array(predicted)

    # 避免除以零的情况
    if np.any(actual == 0):
        raise ValueError("实际值中包含零，无法计算 MAPE。")

    # 计算 MAPE
    mape = np.mean(np.abs((actual - predicted) / actual)) * 100
    return mape
all_predict = pd.DataFrame()
all_feature = pd.DataFrame()
for now_day_pre,now_predict_date in enumerate(pd.date_range(start=unique_dates.max()+ pd.offsets.BDay(32), end=unique_dates.max()+ pd.offsets.BDay(34+23), freq='B')):
    start_time = 32+now_day_pre #用前35+n 到35+n+36天的数据预测当前时刻的数据
    end_time = start_time+36 #
    test_long = 0
    test_start = start_time+test_long
    test_end  = end_time+test_long
    add_column = ['actans','actabn','absActHt','absActSa']
    add_columns =  [f'actans_{i}' for i in range(start_time+1,end_time+1) ] +\
                [f'actabn_{i}' for i in range(start_time+1,end_time+1) ] +\
                [f'absActHt_{i}' for i in range(start_time+1,end_time+1) ] +\
                [f'absActSa_{i}' for i in range(start_time+1,end_time+1) ]
    f_predict = pd.DataFrame()
    
    
    matrix = []

    # 遍历每个时间点
    for time,time1,time2 in zip(unique_times,np.roll(unique_times, shift=-1),np.roll(unique_times, shift=1)):
        # 过滤出当前时间点的数据
        time_data = data[data['time'] == time].set_index('date')['offered']    
        full_dates = pd.date_range(start=unique_dates.min(), end=unique_dates.max(), freq='B')  # 仅工作日
        full_dates = full_dates.difference(holiday_datetime)
        time_data = time_data.groupby(time_data.index).sum()
        time_data = time_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data = pd.concat([time_data.shift(i) for i in range(start_time,end_time)], axis=1)


        add_data = data[data['time'] == time].set_index('date')[add_column]
        add_data = add_data.groupby(add_data.index).sum()
        add_data = add_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        add_data_ = pd.concat([add_data.shift(i) for i in range(start_time,end_time)], axis=1)


        time_data1 = data[data['time'] == time1].set_index('date')['offered']    
        time_data1 = time_data1.groupby(time_data1.index).sum()
        time_data1 = time_data1.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data1 = pd.concat([time_data1.shift(i) for i in range(start_time,end_time)], axis=1)

        time_data2 = data[data['time'] == time2].set_index('date')['offered']    
        time_data2 = time_data2.groupby(time_data2.index).sum()
        time_data2 = time_data2.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data2 = pd.concat([time_data2.shift(i) for i in range(start_time,end_time)], axis=1)


        year_data = time_data.shift(251)


        # 将当前时间点、前半小时和后半小时的数据拼接
        combined_data = pd.concat([time_data,add_data_, shifted_data,shifted_data1,shifted_data2,year_data], axis=1)

        # 创建日期+时间列
        datetime_column = np.array([pd.Timestamp(date) + pd.Timedelta(hours=time.hour, minutes=time.minute) for date in full_dates]).reshape(-1, 1)

        # 将日期+时间列添加到数据中
        combined_data_with_datetime = np.hstack([datetime_column,combined_data.to_numpy()])

        # 将结果存入矩阵
        matrix.append(combined_data_with_datetime)

    max_rows = max(arr.shape[0] for arr in matrix)

    # 将每个时间点的数据填充到最大行数
    for i in range(len(matrix)):
        if matrix[i].shape[0] < max_rows:
            padding = np.full((max_rows - matrix[i].shape[0], 11), np.nan)  # 用NaN填充（10列数据 + 1列时间）
            matrix[i] = np.vstack([matrix[i], padding])

    # 将矩阵堆叠成一个大的二维数组
    matrix = np.vstack(matrix)

    matrix = pd.DataFrame(matrix,columns = ['datetime','offered'] +add_columns +  [f'freq_{i}' for i in range(start_time+1,end_time+1) ]\
                          + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] + \
                          [f'freq_next{i}' for i in range(start_time+1,end_time+1) ]+ \
                           ['years_data']).sort_values('datetime').reset_index(drop = True)

    matrix['year'] = matrix.datetime.map(lambda x:x.year)
    matrix['month'] = matrix.datetime.map(lambda x:x.month)
    matrix['day'] = matrix.datetime.map(lambda x:x.day)
    matrix['hour'] = matrix.datetime.map(lambda x:x.hour)
    matrix['minute'] = matrix.datetime.map(lambda x:x.minute)

    matrix = matrix.sort_values('datetime').reset_index(drop = True)

    read_data = matrix.iloc[:]
    read_data = read_data[~read_data.offered.isna()].reset_index(drop = True)

    fill_na = read_data.iloc[:,1:].apply(lambda x:pd.to_numeric(x))

    fill_na = fill_na.interpolate(method='linear')

    fill_na['datetime'] = read_data.datetime
    fill_na['date'] = fill_na.datetime.dt.date
    fill_na = fill_na.dropna().reset_index(drop = True)

    fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['month'],prefix = 'month').astype(int)],axis = 1)
    fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['year'],prefix = 'year').astype(int)],axis = 1)
    fill_na['week_of_year'] = fill_na['datetime'].dt.isocalendar().week
    fill_na['weekday'] = fill_na['datetime'].dt.day_name()
    fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['week_of_year'],prefix = 'week_of_year').astype(int)],axis = 1)
    fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['weekday'],prefix = 'weekday').astype(int)],axis = 1)

    matrix_predict = []




    # 遍历每个时间点
    for time,time1,time2 in zip(unique_times,np.roll(unique_times, shift=-1),np.roll(unique_times, shift=1)):
        # 过滤出当前时间点的数据
        time_data = data[data['time'] == time].set_index('date')['offered']    
        full_dates = pd.date_range(start=unique_dates.min(), end=unique_dates.max()+ pd.offsets.BDay(start_time), freq='B')  # 仅工作日
        full_dates = full_dates.difference(holiday_datetime)
        time_data = time_data.groupby(time_data.index).sum()    
        time_data = time_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data = pd.concat([time_data.shift(i) for i in range(start_time,end_time)], axis=1)


        add_data = data[data['time'] == time].set_index('date')[add_column]
        add_data = add_data.groupby(add_data.index).sum()
        add_data = add_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        add_data_ = pd.concat([add_data.shift(i) for i in range(start_time,end_time)], axis=1)



        time_data1 = data[data['time'] == time1].set_index('date')['offered'] 
        time_data1 = time_data1.groupby(time_data1.index).sum()
        time_data1 = time_data1.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data1 = pd.concat([time_data1.shift(i) for i in range(start_time,end_time)], axis=1)

        time_data2 = data[data['time'] == time2].set_index('date')['offered']  
        time_data2 = time_data2.groupby(time_data2.index).sum()
        time_data2 = time_data2.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data2 = pd.concat([time_data2.shift(i) for i in range(start_time,end_time)], axis=1)


        year_data = time_data.shift(251)


        # 将当前时间点、前半小时和后半小时的数据拼接
        combined_data = pd.concat([time_data,add_data_, shifted_data,shifted_data1,shifted_data2,year_data], axis=1)

        # 创建日期+时间列
        datetime_column = np.array([pd.Timestamp(date) + pd.Timedelta(hours=time.hour, minutes=time.minute) for date in full_dates]).reshape(-1, 1)

        # 将日期+时间列添加到数据中
        combined_data_with_datetime = np.hstack([datetime_column,combined_data.to_numpy()])[-1:]
        # 将结果存入矩阵
        matrix_predict.append(combined_data_with_datetime)

    max_rows = max(arr.shape[0] for arr in matrix_predict)
    # 将每个时间点的数据填充到最大行数
    for i in range(len(matrix_predict)):
        if matrix_predict[i].shape[0] < max_rows:
            padding = np.full((max_rows - matrix_predict[i].shape[0], 11), np.nan)  # 用NaN填充（10列数据 + 1列时间）
            matrix_predict[i] = np.vstack([matrix_predict[i], padding])

    # 将矩阵堆叠成一个大的二维数组
    matrix_predict = np.vstack(matrix_predict)

    matrix_predict = pd.DataFrame(matrix_predict,columns = ['datetime','offered'] + add_columns+ [f'freq_{i}' for i in range(start_time+1,end_time+1) ]\
                          + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] + [f'freq_next{i}' for i in range(start_time+1,end_time+1) ]+ \
                           ['years_data']).sort_values('datetime').reset_index(drop = True)

    matrix_predict['year'] = matrix_predict.datetime.map(lambda x:x.year)
    matrix_predict['month'] = matrix_predict.datetime.map(lambda x:x.month)
    matrix_predict['day'] = matrix_predict.datetime.map(lambda x:x.day)
    matrix_predict['hour'] = matrix_predict.datetime.map(lambda x:x.hour)
    matrix_predict['minute'] = matrix_predict.datetime.map(lambda x:x.minute)

    matrix_predict = matrix_predict.sort_values('datetime').reset_index(drop = True)

    read_data_predict = matrix_predict.iloc[:]


    fill_na_predict = read_data_predict.iloc[:,2:].apply(lambda x:pd.to_numeric(x))

    fill_na_predict = fill_na_predict.interpolate(method='linear')
    fill_na_predict = fill_na_predict.fillna(1)

    fill_na_predict['datetime'] = read_data_predict.datetime
    fill_na_predict['date'] = now_predict_date

    fill_na_predict = pd.concat([fill_na_predict,pd.get_dummies(fill_na_predict['month'],prefix = 'month').astype(int)],axis = 1)
    fill_na_predict = pd.concat([fill_na_predict,pd.get_dummies(fill_na_predict['year'], prefix = 'year').astype(int)],axis = 1)
    fill_na_predict['week_of_year'] = fill_na_predict['datetime'].dt.isocalendar().week
    fill_na_predict['weekday'] = fill_na_predict['datetime'].dt.day_name()
    fill_na_predict = pd.concat([fill_na_predict,pd.get_dummies(fill_na_predict['week_of_year'],prefix = 'week_of_year').astype(int)],axis = 1)
    fill_na_predict = pd.concat([fill_na_predict,pd.get_dummies(fill_na_predict['weekday'], prefix = 'weekday').astype(int)],axis = 1)
    X_columns = [f'freq_{i}' for i in range(start_time+1,end_time+1) ] + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] +\
                [f'freq_next{i}' for i in range(start_time+1,end_time+1) ] + ['years_data'] + [f'month_{i}' for i in range(1,13)] +\
                [f'year_{i}' for i in np.unique(fill_na.year)] +\
                [f'week_of_year_{i}' for i in fill_na.week_of_year.drop_duplicates().tolist()] +\
                    add_columns

    X_columns2 = [f'freq_{i}' for i in range(start_time+1,end_time+1) ] + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] +\
                [f'freq_next{i}' for i in range(start_time+1,end_time+1) ] + ['years_data'] + [f'month_{i}' for i in range(1,13)] +\
                [f'year_{i}' for i in np.unique(fill_na.year)] +\
                [f'week_of_year_{i}' for i in fill_na.week_of_year.drop_duplicates().tolist()] +\
                    add_columns


    lstm_data = fill_na.dropna(axis = 0)
    for i in X_columns:
        try:
            fill_na_predict[i]
        except:
            fill_na_predict[i] = 0
    lstm_data = lstm_data[lstm_data.year>=2021].reset_index(drop = True)
    week_dict = {'Friday':5, 'Monday':1, 'Tuesday':2, 'Wednesday':3, 'Thursday':4}
    lstm_data['week_num'] = lstm_data['weekday'].map(lambda x:week_dict[x])
    fill_na_predict['week_num'] = fill_na_predict['weekday'].map(lambda x:week_dict[x])


    #for week_ in [1,2,3,4,5]:

    for (hour,minute) in lstm_data[['hour','minute']].drop_duplicates().sort_values(['hour','minute']).to_numpy()[:]:
        now_lstm_data = lstm_data[(lstm_data.hour == hour)&(lstm_data.minute == minute)]
        #now_lstm_data = now_lstm_data[now_lstm_data.week_num == week_]
        now_fill_na_predict = fill_na_predict[(fill_na_predict.hour == hour)&(fill_na_predict.minute == minute)]
        #now_fill_na_predict = now_fill_na_predict[now_fill_na_predict.week_num == week_]

        now_time_list = now_lstm_data.datetime.to_numpy()
        now_lstm_data = now_lstm_data[['offered']+['datetime'] +X_columns].reset_index(drop = True)
        # 归一化数据

        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_data = scaler.fit_transform(now_lstm_data[['offered']][:-test_start])
        # 设置时间步长
        time_step = start_time
        # 创建训练数据集
        X_train_1, y_train =now_lstm_data[X_columns][:-test_start] , scaled_data

        X_test_1, y_test =now_lstm_data[X_columns][-test_start:] , now_lstm_data[['offered']][-test_start:]

        X_train_1 = np.array(X_train_1)
        y_train = np.array(y_train)
        X_test_1 = np.array(X_test_1)
        y_test = np.array(y_test)
        X_test_1 = X_test_1.reshape(X_test_1.shape[0], X_test_1.shape[1], 1)

        # 重塑输入数据为 [samples, time steps, features] 格式
        X_train_1 = X_train_1.reshape(X_train_1.shape[0], X_train_1.shape[1], 1)
        # 构建LSTM模型
        model_1 = Sequential()
        model_1.add(LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
        model_1.add(LSTM(50, return_sequences=False))
        model_1.add(Dense(25))
        model_1.add(Dense(1))

        # 编译模型
        model_1.compile(optimizer='adam', loss='mean_squared_error')
        model_1.fit(X_train_1, y_train, batch_size=32, epochs=20)
        train_predict_1 = model_1.predict(X_train_1)
        train_predict_1 = scaler.inverse_transform(train_predict_1)
        True_data_1 = scaler.inverse_transform(y_train.reshape(-1, 1))
        print(calculate_mape(train_predict_1, True_data_1))
        return_y_1 =  model_1.predict(X_test_1)
        return_y_pred_1 = scaler.inverse_transform(np.array(return_y_1).reshape(-1, 1))
        now_predict_1 = pd.DataFrame(return_y_pred_1,columns = ['predict'])
        now_predict_1['datetime'] = now_time_list[-test_start:]


        now_lstm_data = now_lstm_data[['offered'] +X_columns2].reset_index(drop = True)

        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_data = scaler.fit_transform(now_lstm_data[['offered']][:-test_start])
        # 设置时间步长
        time_step = start_time
        # 创建训练数据集

        X_train_2, y_train =now_lstm_data[X_columns2][:-test_start] , scaled_data

        X_test_2, y_test =now_lstm_data[X_columns2][-test_start:] , now_lstm_data[['offered']][-test_start:]

        X_train_2 = np.array(X_train_2)
        y_train = np.array(y_train)
        X_test_2 = np.array(X_test_2)
        y_test = np.array(y_test)
        X_test_2 = X_test_2.reshape(X_test_2.shape[0], X_test_2.shape[1], 1)

        # 重塑输入数据为 [samples, time steps, features] 格式
        X_train_2 = X_train_2.reshape(X_train_2.shape[0], X_train_2.shape[1], 1)
        # 构建LSTM模型
        model_2 = Sequential()
        model_2.add(LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
        model_2.add(LSTM(50, return_sequences=False))
        model_2.add(Dense(25))
        model_2.add(Dense(1))

        # 编译模型
        model_2.compile(optimizer='adam', loss='mean_squared_error')
        model_2.fit(X_train_2, y_train, batch_size=32, epochs=20)
        train_predict_2 = model_2.predict(X_train_2)
        train_predict_2 = scaler.inverse_transform(train_predict_2)
        True_data_2 = scaler.inverse_transform(y_train.reshape(-1, 1))
        print(calculate_mape(train_predict_2, True_data_2))
        return_y_2 =  model_2.predict(X_test_2)
        return_y_pred_2 = scaler.inverse_transform(np.array(return_y_2).reshape(-1, 1))
        now_predict_2 = pd.DataFrame(return_y_pred_2,columns = ['predict'])
        now_predict_2['datetime'] = now_time_list[-test_start:]
        print(calculate_mape(y_test, return_y_pred_2))
        if calculate_mape(y_test, return_y_pred_1) < calculate_mape(y_test, return_y_pred_2):


            f_predict = pd.concat([now_predict_1,f_predict])
            future_X = now_fill_na_predict[X_columns].to_numpy()
            future_data = scaler.inverse_transform(np.array(\
                            model_1.predict(future_X.reshape(future_X.shape[0], future_X.shape[1], 1))).reshape(-1, 1))
            future_data = pd.DataFrame(future_data,columns = ['predict'])
            future_data['day'] = now_predict_date
            future_data['hour'] = hour
            future_data['minute'] = minute
            all_feature = pd.concat([all_feature,future_data])
        else:
            f_predict = pd.concat([now_predict_2,f_predict])
            future_X = now_fill_na_predict[X_columns2].to_numpy()
            future_data = scaler.inverse_transform(np.array(\
                            model_2.predict(future_X.reshape(future_X.shape[0], future_X.shape[1], 1))).reshape(-1, 1))
            future_data = pd.DataFrame(future_data,columns = ['predict'])
            future_data['day'] = now_predict_date
            future_data['hour'] = hour
            future_data['minute'] = minute
            all_feature = pd.concat([all_feature,future_data])
    f_predict = pd.merge(f_predict,lstm_data[['offered','datetime']],on = ['datetime'])
    f_predict = f_predict.sort_values('datetime').reset_index(drop = True)
    all_predict = pd.concat([all_predict,f_predict])
    print(all_predict.head(2))
all_predict = pd.merge(all_predict,lstm_data[['offered','datetime']],on = ['datetime'])
all_predict = all_predict.sort_values('datetime').reset_index(drop = True)
all_predict['diff'] = (all_predict.predict - all_predict.offered)
all_predict['hour'] = all_predict.datetime.dt.hour
all_predict.to_excel(r'workspace/RS_KSR_Test_Daily_{}.xlsx'.format(month_number))
time_ = []
for (hour,minute) in fill_na_predict[['hour','minute']].drop_duplicates().to_numpy()[:]:
    now_fill_na_predict = fill_na_predict[(fill_na_predict.hour == hour)&(fill_na_predict.minute == minute)]
    time_.append(now_fill_na_predict.datetime.to_numpy())
all_feature['datetime'] =  np.hstack(time_)
all_feature.predict = all_feature.predict.map(lambda x:max(0,x))
data_['conversation_start_interval_tmst']=pd.to_datetime(data_['conversation_start_interval_tmst'])
all_feature['datetime']=pd.to_datetime(all_feature['datetime'])
all_feature = pd.merge(all_feature,data_[['offered','conversation_start_interval_tmst']],left_on = ['datetime'],right_on=['conversation_start_interval_tmst'])
all_feature = all_feature.sort_values('datetime').reset_index(drop = True)
all_feature['mape']=np.abs(all_feature['offered']-all_feature['predict'])/(all_feature['offered'])*100
all_feature.to_excel(r'workspace/RS_KSR_Taget_Daily_{}.xlsx'.format(month_number))
